In [1]:
import numpy as np
import pandas as pd
import pybithumb
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from Funcs_CNN4 import obv, macd, cmo, rsi
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)

Scaler = MinMaxScaler()

dir = 'ohlcv/'
ohlcv_list = os.listdir(dir)


def min_max_scaler(price):
    Scaler = MinMaxScaler()
    Scaler.fit(price)

    return Scaler.transform(price)


def low_high(Coin, input_data_length, ip_limit=None, trade_limit=None, crop_size=300, lowhigh_point=None):

    #   거래 제한은 고점과 저점을 분리한다.

    #   User-Agent Configuration
    #   IP - Change
    if ip_limit is None:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')
    else:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1', 'proxyon')

    price_gap = ohlcv_excel.close.max() / ohlcv_excel.close.min()
    if (price_gap < 1.07) and (trade_limit is not None):
        return None, None, None, None

    ohlcv_excel['OBV'] = obv(ohlcv_excel)

    if lowhigh_point is not None:
        check_span = 20
        trade_state = [np.NaN] * len(ohlcv_excel)
        for i in range(check_span, len(ohlcv_excel) - check_span):
            #   저점
            if ohlcv_excel['close'][i - check_span:i].min() >= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i + 1:i + 1 + check_span].min() >= ohlcv_excel['close'][i]:
                    # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[0] <= 3:
                    if 1 not in trade_state[i - check_span:i]:
                        trade_state[i] = 1
                    else:
                        trade_state[i] = 0
                    # else:
                    #     trade_state[i] = 0
                else:
                    trade_state[i] = 0
            #   고점
            elif ohlcv_excel['close'][i - check_span:i].max() <= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i + 1:i + 1 + check_span].max() <= ohlcv_excel['close'][i]:
                    # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[-1] <= 3:
                    if 2 not in trade_state[i - check_span:i]:
                        trade_state[i] = 2
                    else:
                        trade_state[i] = 0
                # else:
                #     trade_state[i] = 0
                else:
                    trade_state[i] = 0

            #   거래 안함
            else:
                trade_state[i] = 0

        ohlcv_excel['trade_state'] = trade_state

        #   현재가가 이전 저점보다 0.1 이상 크지 않으면 거래하지 않는다.
        low_index = None
        for i in range(len(trade_state)):
            if trade_state[-i] == 1:
                low_index = i

    closeprice = ohlcv_excel['close'].iloc[-1]
    datetime_list = ohlcv_excel.index.values

    # ----------- dataX, dataY 추출하기 -----------#
    #   OBV :
    ohlcv_data = ohlcv_excel.values[1:].astype(np.float)

    scaled_close = min_max_scaler(ohlcv_data[:, [1]])
    prev_low = scaled_close[-low_index]
    pres_low = scaled_close[-1]
    if pres_low - prev_low < 0.1:
        return None, None, None, None

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        # price = ohlcv_data[:, :4]
        # volume = ohlcv_data[:, [4]]
        # OBV = ohlcv_data[:, [-1]]
        #
        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_OBV = min_max_scaler(OBV)
        # print(scaled_MA60.shape)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # print(x.shape)
        # print(ohlcv_data.shape)
        # quit()

        # if (x[-1][1] > 0.3) and (trade_limit is not None):
        #     return None, None

        # print(x.shape)  # (258, 6)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        for i in range(crop_size, len(ohlcv_data) + 1):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            scaled_price = min_max_scaler(group_x[:, :4])
            # scaled_volume = min_max_scaler(group_x[:, [4]])
            # scaled_OBV = min_max_scaler(group_x[:, [-1]])
            # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1) + sudden_death  # axis=1, 세로로 합친다
            x = scaled_price  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]
            # print(group_x[-1:, 1:2])
            # print(group_x.shape)
            # print(group_x.max())
            dataX.append(group_x)  # dataX 리스트에 추가

        if len(dataX) < 100:
            return None, None, None, None

        # quit()
        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # closeprice = np.roll(np.array(list(map(lambda x: x[-1][[1]][0], X_test))), -1)
        # plt.plot(closeprice)
        # plt.show()

        return X_test, closeprice, min_max_scaler(ohlcv_data[crop_size:, [1]]), datetime_list


def made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=None, sudden_death=0):

    ohlcv_excel = pd.read_excel(dir + file, index_col=0)

    ohlcv_excel['MA20'] = ohlcv_excel['close'].rolling(20).mean()
    ohlcv_excel['CMO'] = cmo(ohlcv_excel)
    ohlcv_excel['OBV'] = obv(ohlcv_excel)
    ohlcv_excel['RSI'] = rsi(ohlcv_excel)
    macd(ohlcv_excel)

    # print(ohlcv_excel)
    # quit()

    #   이후 check_span 데이터와 현재 포인트를 비교해서 현재 포인트가 저가인지 고가인지 예측한다.
    #   진입, 저점, 고점, 거래 안함의 y_label 인 trade_state  >> [1, 2, 0]
    #   저점과 고점은 최대 3개의 중복 값을 허용한다.
    trade_state = [np.NaN] * len(ohlcv_excel)
    for i in range(len(ohlcv_excel) - check_span):
        #   저점
        if ohlcv_excel['MA20'][i + 1:i + 1 + check_span].min() >= ohlcv_excel['MA20'][i]:
            if ohlcv_excel['MA20'][i:i + 1 + check_span].value_counts().sort_index().iloc[0] <= 3:
                trade_state[i] = 1
            else:
                trade_state[i] = 0
        #   고점
        elif ohlcv_excel['MA20'][i + 1:i + 1 + check_span].max() <= ohlcv_excel['MA20'][i]:
            if ohlcv_excel['MA20'][i:i + 1 + check_span].value_counts().sort_index().iloc[-1] <= 3:
                trade_state[i] = 2
            else:
                trade_state[i] = 0
        #   거래 안함
        else:
            trade_state[i] = 0

    ohlcv_excel['trade_state'] = trade_state

    # ----------- dataX, dataY 추출하기 -----------#
    # print(ohlcv_excel.info())
    # quit()
    # ohlcv_excel.to_excel('test.xlsx')

    # NaN 제외하고 데이터 자르기 (데이터가 PIXEL 로 들어간다고 생각하면 된다)
    #   OBV : -CHECK_SPAN
    ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.MACD_Signal.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[1: -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.CMO.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.RSI.isna()): -check_span].astype(np.float)

    # print(pd.DataFrame(ohlcv_data).info())
    # print(pd.DataFrame(ohlcv_data).to_excel('test.xlsx'))
    # print(list(map(float, ohlcv_data[0])))
    # quit()

    #       출력되는 Y 데이터 길이를 동일화 시킨다.     #
    # ohlcv_data = ohlcv_data[-300:]

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
#         price = ohlcv_data[:, :4]
#         volume = ohlcv_data[:, [4]]
#         MA20 = ohlcv_data[:, [-8]]
#         CMO = ohlcv_data[:, [-7]]
#         OBV = ohlcv_data[:, [-6]]
#         RSI = ohlcv_data[:, [-5]]
#         MACD = ohlcv_data[:, [-4]]
#         MACD_Signal = ohlcv_data[:, [-3]]
#         MACD_OSC = ohlcv_data[:, [-2]]

        #   Flexible Y_data    #
        trade_state = ohlcv_data[:, [-1]]

        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_CMO = min_max_scaler(CMO)
        # scaled_OBV = min_max_scaler(OBV)
        # scaled_RSI = min_max_scaler(RSI)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # x = scaled_price
        y = trade_state
        # print(x.shape, y_low.shape)  # (258, 6) (258, 1)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        dataY = []  # Target 을 담을 그릇
        for i in range(crop_size, len(ohlcv_data)):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            group_y = y[i]
            price = group_x[:, :4]
            volume = group_x[:, [4]]
            MA20 = group_x[:, [-8]]
            CMO = group_x[:, [-7]]
            OBV = group_x[:, [-6]]
            RSI = group_x[:, [-5]]
            MACD = group_x[:, [-4]]
            MACD_Signal = group_x[:, [-3]]
            MACD_OSC = group_x[:, [-2]]
            scaled_price = min_max_scaler(price)
            scaled_volume = min_max_scaler(volume)
            scaled_MA20 = min_max_scaler(MA20)
            scaled_CMO = min_max_scaler(CMO)
            scaled_OBV = min_max_scaler(OBV)
            scaled_RSI = min_max_scaler(RSI)
            scaled_MACD = min_max_scaler(MACD)
            scaled_MACD_Signal = min_max_scaler(MACD_Signal)
            scaled_MACD_OSC = min_max_scaler(MACD_OSC)
            x = np.concatenate((scaled_price, scaled_volume, scaled_MA20, scaled_CMO, scaled_OBV, scaled_RSI,
                                scaled_MACD, scaled_MACD_Signal, scaled_MACD_OSC), axis=1)
            # x = scaled_x + sudden_death  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]
            # print(group_x[0])
            # quit()

            #   데이터 값에 결측치가 존재하는 경우 #
            if sum(sum(np.isnan(group_x))) > 0:
                return None, None, None

            dataX.append(group_x)  # dataX 리스트에 추가
            dataY.append(group_y)

        if len(dataX) < 100:
            print('len(dataX) < 100')
            return None, None, None

        # X_test = np.array(dataX)
        # row = X_test.shape[1]
        # col = X_test.shape[2]
        #
        # X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # print(X_test.shape)

        #       Exstracting fiexd X_data       #
        sliced_ohlcv = ohlcv_data[crop_size:, :x.shape[1]]

        #                      Get Figure                     #
        if get_fig == 1:
            spanlist_low = []
            spanlist_high = []

            for m in range(len(trade_state)):
                if (trade_state[m] > 0.5) and (trade_state[m] < 1.5):
                    if m + 1 < len(trade_state):
                        spanlist_low.append((m, m + 1))
                    else:
                        spanlist_low.append((m - 1, m))

            for m in range(len(trade_state)):
                if (trade_state[m] > 1.5) and (trade_state[m] < 2.5):
                    if m + 1 < len(trade_state):
                        spanlist_high.append((m, m + 1))
                    else:
                        spanlist_high.append((m - 1, m))

            # ----------- 인덱스 초기화 됨 -----------#

            # ----------- 공통된 Chart 그리기 -----------#

            plt.subplot(211)
            plt.plot(min_max_scaler(ohlcv_data[:, [5]]), 'gold', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_low)):
                plt.axvspan(spanlist_low[i][0], spanlist_low[i][1], facecolor='c', alpha=0.5)

            plt.subplot(212)
            plt.plot(min_max_scaler(ohlcv_data[:, [5]]), 'gold', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_high)):
                plt.axvspan(spanlist_high[i][0], spanlist_high[i][1], facecolor='m', alpha=0.5)

            Date = file.split()[0]
            Coin = file.split()[1].split('.')[0]
            plt.savefig('./Figure_data/%s_%s/%s %s.png' % (input_data_length, model_num, Date, Coin), dpi=500)
            plt.close()
            # plt.show()
            # ----------- Chart 그리기 -----------#

        return dataX, dataY, sliced_ohlcv


if __name__ == '__main__':

    # ----------- Params -----------#
    input_data_length = 100
    model_num = 52

    #       Make folder      #
    try:
        os.mkdir('./Figure_data/%s_%s/' % (input_data_length, model_num))

    except Exception as e:
        pass
    check_span = 30
    get_fig = 0

    Made_X = []
    Made_Y = []

#     ohlcv_list = ['2020-01-10 ETH ohlcv.xlsx']

    for file in ohlcv_list:
        try:
            if int(file.split()[0].split('-')[1]) != 1:
                continue


            result = made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=input_data_length)
            # result = low_high('dvp'.upper(), input_data_length)
            # print(result)
            # quit()

            # ------------ 데이터가 있으면 dataX, dataY 병합하기 ------------#
            if result is not None:
                if result[0] is not None:

                    Made_X += result[0]
                    Made_Y += result[1]

                # 누적 데이터량 표시
                print(file, len(Made_X))
                
        except Exception as e:
            pass

    # SAVING X, Y
    X = np.array(Made_X)
    Y = np.array(Made_Y)
    np.save('./Made_X/Made_X %s_%s' % (input_data_length, model_num), X)
    np.save('./Made_X/Made_Y %s_%s' % (input_data_length, model_num), Y)
    # np.save('./Made_X_high/Made_Y %s_%s' % (input_data_length, model_num), Y_high)



2020-01-16 XLM ohlcv.xlsx 929
2020-01-23 TRX ohlcv.xlsx 929
2020-01-31 AMO ohlcv.xlsx 1505
2020-01-16 SNT ohlcv.xlsx 1646
len(dataX) < 100
2020-01-19 LBA ohlcv.xlsx 1646
2020-01-11 FX ohlcv.xlsx 1888
2020-01-22 LUNA ohlcv.xlsx 1888
2020-01-16 LINK ohlcv.xlsx 2808
len(dataX) < 100
2020-01-28 PLY ohlcv.xlsx 2808
2020-01-13 FCT ohlcv.xlsx 2939
2020-01-20 IPX ohlcv.xlsx 2939
len(dataX) < 100
2020-01-27 ITC ohlcv.xlsx 2939
len(dataX) < 100
2020-01-24 BZNT ohlcv.xlsx 2939
2020-01-20 SNT ohlcv.xlsx 3115
2020-01-29 TMTG ohlcv.xlsx 3953
len(dataX) < 100
2020-01-25 VET ohlcv.xlsx 3953
2020-01-21 EOS ohlcv.xlsx 5039
len(dataX) < 100
2020-01-24 FAB ohlcv.xlsx 5039
2020-01-22 TRV ohlcv.xlsx 5154
len(dataX) < 100
2020-01-26 DVP ohlcv.xlsx 5154
2020-01-22 MTL ohlcv.xlsx 5455
len(dataX) < 100
2020-01-12 POWR ohlcv.xlsx 5455
2020-01-28 TRX ohlcv.xlsx 5455
2020-01-31 LAMB ohlcv.xlsx 5455
len(dataX) < 100
2020-01-12 MCO ohlcv.xlsx 5455
len(dataX) < 100
2020-01-20 CMT ohlcv.xlsx 5455
2020-01-22 BTG ohlcv.

len(dataX) < 100
2020-01-28 VET ohlcv.xlsx 42434
2020-01-30 BTC ohlcv.xlsx 43698
len(dataX) < 100
2020-01-31 XMR ohlcv.xlsx 43698
2020-01-13 DAD ohlcv.xlsx 44070
len(dataX) < 100
2020-01-25 OMG ohlcv.xlsx 44070
2020-01-20 MTL ohlcv.xlsx 44769
len(dataX) < 100
2020-01-15 FZZ ohlcv.xlsx 44769
len(dataX) < 100
2020-01-19 FX ohlcv.xlsx 44769
len(dataX) < 100
2020-01-12 ELF ohlcv.xlsx 44769
2020-01-20 BTT ohlcv.xlsx 45014
2020-01-27 XSR ohlcv.xlsx 45720
len(dataX) < 100
2020-01-20 OGO ohlcv.xlsx 45720
len(dataX) < 100
2020-01-20 LRC ohlcv.xlsx 45720
len(dataX) < 100
2020-01-31 GNT ohlcv.xlsx 45720
len(dataX) < 100
2020-01-19 LOOM ohlcv.xlsx 45720
len(dataX) < 100
2020-01-12 CTXC ohlcv.xlsx 45720
2020-01-19 LTC ohlcv.xlsx 46562
2020-01-15 ZEC ohlcv.xlsx 47034
2020-01-30 REP ohlcv.xlsx 47034
len(dataX) < 100
2020-01-12 ITC ohlcv.xlsx 47034
len(dataX) < 100
2020-01-29 WET ohlcv.xlsx 47034
2020-01-19 INS ohlcv.xlsx 47382
2020-01-24 TMTG ohlcv.xlsx 48393
len(dataX) < 100
2020-01-20 WICC ohlcv.xl

2020-01-29 STEEM ohlcv.xlsx 96926
2020-01-30 ADA ohlcv.xlsx 97874
len(dataX) < 100
2020-01-30 CRO ohlcv.xlsx 97874
2020-01-23 VALOR ohlcv.xlsx 98054
len(dataX) < 100
2020-01-11 REP ohlcv.xlsx 98054
len(dataX) < 100
2020-01-19 ZIL ohlcv.xlsx 98054
2020-01-26 CON ohlcv.xlsx 98054
2020-01-20 XRP ohlcv.xlsx 99302
2020-01-25 BTG ohlcv.xlsx 99991
2020-01-24 XEM ohlcv.xlsx 100220
2020-01-21 WPX ohlcv.xlsx 100972
len(dataX) < 100
2020-01-28 LOOM ohlcv.xlsx 100972
len(dataX) < 100
2020-01-14 PAY ohlcv.xlsx 100972
len(dataX) < 100
2020-01-23 ITC ohlcv.xlsx 100972
len(dataX) < 100
2020-01-19 CMT ohlcv.xlsx 100972
2020-01-29 WTC ohlcv.xlsx 100972
2020-01-27 DASH ohlcv.xlsx 101148
2020-01-27 XRP ohlcv.xlsx 101148
2020-01-25 BCD ohlcv.xlsx 101549
len(dataX) < 100
2020-01-15 APIS ohlcv.xlsx 101549
2020-01-31 OGO ohlcv.xlsx 102350
2020-01-31 FCT ohlcv.xlsx 102711
len(dataX) < 100
2020-01-29 ABT ohlcv.xlsx 102711
2020-01-16 LUNA ohlcv.xlsx 102711
len(dataX) < 100
2020-01-22 CTXC ohlcv.xlsx 102711
len(d

2020-01-25 APIS ohlcv.xlsx 132751
2020-01-11 DAD ohlcv.xlsx 133547
len(dataX) < 100
2020-01-28 ETZ ohlcv.xlsx 133547
2020-01-22 OCN ohlcv.xlsx 133547
2020-01-31 TMTG ohlcv.xlsx 134489
2020-01-11 CON ohlcv.xlsx 135376
len(dataX) < 100
2020-01-22 XMR ohlcv.xlsx 135376
2020-01-28 LTC ohlcv.xlsx 135376
len(dataX) < 100
2020-01-13 LBA ohlcv.xlsx 135376
2020-01-16 LTC ohlcv.xlsx 136397
2020-01-15 AMO ohlcv.xlsx 136397
len(dataX) < 100
2020-01-26 FX ohlcv.xlsx 136397
2020-01-24 LUNA ohlcv.xlsx 136397
len(dataX) < 100
2020-01-30 MXC ohlcv.xlsx 136397
2020-01-14 ABT ohlcv.xlsx 137030
len(dataX) < 100
2020-01-27 IOST ohlcv.xlsx 137030
2020-01-19 IOST ohlcv.xlsx 137030
2020-01-20 WOM ohlcv.xlsx 137137
2020-01-11 GNT ohlcv.xlsx 137137
2020-01-29 MTL ohlcv.xlsx 137469
2020-01-20 ELF ohlcv.xlsx 137606
2020-01-14 ZEC ohlcv.xlsx 137792
2020-01-30 BTG ohlcv.xlsx 138749
2020-01-21 LTC ohlcv.xlsx 138749
len(dataX) < 100
2020-01-12 ANKR ohlcv.xlsx 138749
len(dataX) < 100
2020-01-31 ABT ohlcv.xlsx 138749
l

len(dataX) < 100
2020-01-15 DVP ohlcv.xlsx 174170
len(dataX) < 100
2020-01-24 VET ohlcv.xlsx 174170
2020-01-16 ELF ohlcv.xlsx 174463
2020-01-14 RNT ohlcv.xlsx 174585
2020-01-28 ETH ohlcv.xlsx 175589
len(dataX) < 100
2020-01-24 SNT ohlcv.xlsx 175589
len(dataX) < 100
2020-01-29 GXC ohlcv.xlsx 175589
len(dataX) < 100
2020-01-24 IOST ohlcv.xlsx 175589
len(dataX) < 100
2020-01-22 ETZ ohlcv.xlsx 175589
2020-01-19 ENJ ohlcv.xlsx 175589
len(dataX) < 100
2020-01-27 LAMB ohlcv.xlsx 175589
2020-01-25 TMTG ohlcv.xlsx 176285
len(dataX) < 100
2020-01-28 DVP ohlcv.xlsx 176285
2020-01-16 WOM ohlcv.xlsx 177216
len(dataX) < 100
2020-01-29 PLY ohlcv.xlsx 177216
len(dataX) < 100
2020-01-28 HDAC ohlcv.xlsx 177216
2020-01-16 FNB ohlcv.xlsx 178194
len(dataX) < 100
2020-01-24 LAMB ohlcv.xlsx 178194
2020-01-15 BCH ohlcv.xlsx 179170
2020-01-20 MBL ohlcv.xlsx 179170
2020-01-22 FCT ohlcv.xlsx 179170
len(dataX) < 100
2020-01-16 MBL ohlcv.xlsx 179170
len(dataX) < 100
2020-01-25 ZRX ohlcv.xlsx 179170
2020-01-15 BTC 

len(dataX) < 100
2020-01-23 RNT ohlcv.xlsx 213294
len(dataX) < 100
2020-01-29 ELF ohlcv.xlsx 213294
2020-01-16 WAVES ohlcv.xlsx 213294
2020-01-19 XLM ohlcv.xlsx 214243
len(dataX) < 100
2020-01-21 XMR ohlcv.xlsx 214243
len(dataX) < 100
2020-01-25 CHR ohlcv.xlsx 214243
2020-01-21 FCT ohlcv.xlsx 214456
2020-01-24 ETH ohlcv.xlsx 214456
len(dataX) < 100
2020-01-26 PLY ohlcv.xlsx 214456
len(dataX) < 100
2020-01-22 LOOM ohlcv.xlsx 214456
2020-01-23 XLM ohlcv.xlsx 214848
len(dataX) < 100
2020-01-28 ABT ohlcv.xlsx 214848
len(dataX) < 100
2020-01-23 OMG ohlcv.xlsx 214848
len(dataX) < 100
2020-01-27 VALOR ohlcv.xlsx 214848
2020-01-16 FCT ohlcv.xlsx 215435
len(dataX) < 100
2020-01-24 LOOM ohlcv.xlsx 215435
2020-01-12 ETH ohlcv.xlsx 216540
len(dataX) < 100
2020-01-15 ANKR ohlcv.xlsx 216540
len(dataX) < 100
2020-01-16 MCO ohlcv.xlsx 216540
len(dataX) < 100
2020-01-29 FAB ohlcv.xlsx 216540
2020-01-21 XRP ohlcv.xlsx 217730
len(dataX) < 100
2020-01-21 LAMB ohlcv.xlsx 217730
len(dataX) < 100
2020-01-30 

2020-01-11 LUNA ohlcv.xlsx 255652
len(dataX) < 100
2020-01-25 SNT ohlcv.xlsx 255652
2020-01-28 DASH ohlcv.xlsx 255837
len(dataX) < 100
2020-01-30 VALOR ohlcv.xlsx 255837
2020-01-24 XRP ohlcv.xlsx 256950
2020-01-15 DASH ohlcv.xlsx 257514
len(dataX) < 100
2020-01-29 VALOR ohlcv.xlsx 257514
2020-01-24 ENJ ohlcv.xlsx 257967
len(dataX) < 100
2020-01-15 OCN ohlcv.xlsx 257967
2020-01-26 REP ohlcv.xlsx 258621
2020-01-16 ICX ohlcv.xlsx 258850
2020-01-22 DASH ohlcv.xlsx 259076
2020-01-12 ETZ ohlcv.xlsx 259736
2020-01-31 BTT ohlcv.xlsx 259965
len(dataX) < 100
2020-01-14 LRC ohlcv.xlsx 259965
len(dataX) < 100
2020-01-14 FX ohlcv.xlsx 259965
2020-01-16 PAY ohlcv.xlsx 260080
2020-01-19 DAC ohlcv.xlsx 260080
len(dataX) < 100
2020-01-25 MCO ohlcv.xlsx 260080
len(dataX) < 100
2020-01-28 ITC ohlcv.xlsx 260080
len(dataX) < 100
2020-01-29 RDN ohlcv.xlsx 260080
len(dataX) < 100
2020-01-20 TRUE ohlcv.xlsx 260080
len(dataX) < 100
2020-01-13 WTC ohlcv.xlsx 260080
2020-01-22 TMTG ohlcv.xlsx 260080
len(dataX) <

2020-01-13 ETZ ohlcv.xlsx 295164
len(dataX) < 100
2020-01-24 ICX ohlcv.xlsx 295164
2020-01-30 WAVES ohlcv.xlsx 295164
len(dataX) < 100
2020-01-11 FZZ ohlcv.xlsx 295164
len(dataX) < 100
2020-01-26 TRUE ohlcv.xlsx 295164
len(dataX) < 100
2020-01-31 ITC ohlcv.xlsx 295164
2020-01-12 HYC ohlcv.xlsx 295358
2020-01-23 FX ohlcv.xlsx 295486
2020-01-20 FNB ohlcv.xlsx 295889
2020-01-28 ICX ohlcv.xlsx 296461
len(dataX) < 100
2020-01-23 DVP ohlcv.xlsx 296461
len(dataX) < 100
2020-01-24 QKC ohlcv.xlsx 296461
2020-01-20 DAC ohlcv.xlsx 297668
2020-01-26 BTG ohlcv.xlsx 298339
2020-01-28 BTG ohlcv.xlsx 299467
len(dataX) < 100
2020-01-21 STRAT ohlcv.xlsx 299467
len(dataX) < 100
2020-01-31 WET ohlcv.xlsx 299467
2020-01-21 ETH ohlcv.xlsx 300570
len(dataX) < 100
2020-01-22 WAXP ohlcv.xlsx 300570
2020-01-19 XSR ohlcv.xlsx 300570
2020-01-21 FNB ohlcv.xlsx 300873
2020-01-15 LTC ohlcv.xlsx 301602
2020-01-14 DAD ohlcv.xlsx 301812
len(dataX) < 100
2020-01-30 WAXP ohlcv.xlsx 301812
2020-01-21 VALOR ohlcv.xlsx 3021

len(dataX) < 100
2020-01-30 ETZ ohlcv.xlsx 337941
2020-01-15 BCD ohlcv.xlsx 338749
len(dataX) < 100
2020-01-24 AE ohlcv.xlsx 338749
2020-01-31 LINK ohlcv.xlsx 339267
2020-01-12 ETC ohlcv.xlsx 339877
2020-01-31 MTL ohlcv.xlsx 340180
2020-01-24 ADA ohlcv.xlsx 340522
len(dataX) < 100
2020-01-31 APIS ohlcv.xlsx 340522
2020-01-21 INS ohlcv.xlsx 340742
len(dataX) < 100
2020-01-30 AE ohlcv.xlsx 340742
len(dataX) < 100
2020-01-29 TRV ohlcv.xlsx 340742
2020-01-20 WAXP ohlcv.xlsx 340742
len(dataX) < 100
2020-01-30 DAD ohlcv.xlsx 340742
2020-01-25 IPX ohlcv.xlsx 340742
len(dataX) < 100
2020-01-13 MCO ohlcv.xlsx 340742
len(dataX) < 100
2020-01-25 CMT ohlcv.xlsx 340742
len(dataX) < 100
2020-01-25 ITC ohlcv.xlsx 340742
len(dataX) < 100
2020-01-25 ABT ohlcv.xlsx 340742
2020-01-24 GNT ohlcv.xlsx 341038
len(dataX) < 100
2020-01-25 DAD ohlcv.xlsx 341038
2020-01-23 ETZ ohlcv.xlsx 341267
len(dataX) < 100
2020-01-24 TRV ohlcv.xlsx 341267
len(dataX) < 100
2020-01-12 WAVES ohlcv.xlsx 341267
len(dataX) < 100
